In [ ]:
# Load the python package
import os
from dynetan.toolkit import *
from dynetan.viz import *
from dynetan.proctraj import *
from dynetan.gencor import *
from dynetan.contact import *
from dynetan.datastorage import *

from MDAnalysis.analysis import distances as MDAdistances
#from numpy.linalg import norm
#from itertools import islice
from itertools import combinations
from scipy import stats
from collections import OrderedDict

import networkx as nx
import numpy as np
import scipy as sp

In [ ]:
# For visualization
from bokeh.io import output_file, output_notebook, push_notebook, show
from bokeh import models as bokehModels
from bokeh import transform as bokehTransform
from bokeh import layouts as bokehLayouts
from bokeh import plotting as bokehPlotting
from bokeh import palettes as bokehPalettes
from bokeh import events as bokehEvents
# For pre-calculating CArtesian distances based on 2D embedding
from sklearn.manifold import MDS

In [ ]:
mapResidueNames={'ALA':'A','CYS':'C','ASP':'D','GLU':'E','PHE':'F',
                 'GLY':'G','HIS':'H','HSD':'H','HSE':'H','ILE':'I','LYS':'K','LEU':'L',
                 'MET':'M','ASN':'N','PRO':'P','GLN':'Q','ARG':'R',
                 'SER':'S','THR':'T','VAL':'V','TRP':'W','TYR':'Y',
                 'MG':'Mg','ATP':'Atp','POPC':'Popc','SOL':'h2o'}

def name_node(dnad, node):
    #i=dnad.nodesAtmSel[node].index
    resname=dnad.nodesAtmSel[node].resname ; resid=dnad.nodesAtmSel[node].resid
    return "%s%s" % (mapResidueNames[resname], resid)

def clarify_duplicate_nodes(dictNames, dictSuffix):
    """
    From two dicts with the same keys, add the respective suffix to all keys in the former that possess duplicate values.
# For visualization
from bokeh.io import output_file, output_notebook, push_notebook, show
from bokeh import models as bokehModels
from bokeh import transform as bokehTransform
from bokeh import layouts as bokehLayouts
from bokeh import plotting as bokehPlotting
from bokeh import palettes as bokehPalettes
from bokeh import events as bokehEvents
# For pre-calculating CArtesian distances based on 2D embedding
from sklearn.manifold import MDS    """
    from itertools import chain
    dictRev = {}
    for k, v in dictNames.items():
        dictRev.setdefault(v, set()).add(k)
        setDuplicateKeys = set(chain.from_iterable( v for k, v in dictRev.items() if len(v) > 1))
    for k in setDuplicateKeys:
        dictNames[k] = dictNames[k]+"_"+dictSuffix[k]
    return dictNames  

In [ ]:
def encode_neighbour_information(G):
    # = = Do over each node. First enccode javascript ID, assuming that order is preserved.
    for i, x in enumerate(G.nodes()):
        G.nodes[x]['jsID']=i
    for i, x in enumerate(G.edges()):
        G.edges[x]['jsID']=i
        
    for x in G.nodes():
        G.nodes[x]['jsNeighbours'] = [ G.nodes[y]['jsID'] for y in G.neighbors(x) ]
#        G.nodes[x]['jsEdges'] = [ G.edges[y]['jsID'] for y in G.edges(x) ]

In [ ]:
def simplify_graph(G, attr='segid'):
    outG = nx.Graph()

    for n,d in G.nodes(data=attr):
        if d not in outG.nodes():
            outG.add_node( d )
    for u,v,w in G.edges(data='weight'):
        k1 = G.nodes[u][attr]
        k2 = G.nodes[v][attr]
        if (k1,k2) not in outG.edges():
            outG.add_edge(k1,k2, weight=0.0)
        outG.edges[k1,k2]['weight'] += w
    return outG
    # Hello World

In [ ]:
bPythonExport = False

In [ ]:
if not bPythonExport:
    %cd ..
    # Define mutant file IO locations. wt, P67L, E56K, R75Q, S945L, dF508
    allele="dF508" ; temperature="310K"
    dataDir = "./results/%s/%s/" % (allele, temperature)
    #Path where results will be written (you may want plots and data files in a new location)
    workDir = "./results/%s/%s/analysis" % (allele, temperature)
    fileNameRoot = "1to3"
    fullPathRoot = os.path.join(dataDir, fileNameRoot)
    
    #outputFileName = "./results/networkView_%s_%s.html" % (allele, temperature)
    outputFileName = "./networkCompare.html"
    titleGraph     = "%s %s simulations" % (allele, temperature)
    
    fileImportPos  = './CFTRGraphReferencePositions.txt'
    fileExportPos  = './temp.txt'
    #fileClusterDefinitions = None
    fileClusterDefinitions = './Stable_Solvent_Clustering.cluster_definitions_d3.5_r0.50.txt'
    
    bIncludeReference = False
    if allele != "wt":
        dataWTDir = "./results/wt/%s/" % (temperature)
        fileNameRoot = "1to6"
        fullPathWTRoot = os.path.join(dataWTDir, fileNameRoot)
        bIncludeReference = True

In [ ]:
print("= = = Loading input graph data...")
dnad = DNAdata()
# = = = loadFromFile will automatically output debug lines.
dnad.loadFromFile(fullPathRoot)
dcdVizFile = fullPathRoot + "_reducedTraj.dcd"
pdbVizFile = fullPathRoot + "_reducedTraj.pdb"
mdU = mda.Universe(pdbVizFile,dcdVizFile)
dnad.nodesAtmSel = mdU.atoms[ dnad.nodesIxArray ]

In [ ]:
G = simplify_graph(dnad.nxGraphs[0], 'segid')

In [ ]:
def arrange_self_edges(G, CDS, pos):
    """
    Sets on a ColumnDataSource to plot graph self-edges.
    Runs a simplistic computation over neighbour node positions such that the edge will be oriented to lesson overlap with existint edge rays.
    """
    widthWedge=np.pi/4
    for u,v,weight in G.edges(data='weight'):
        if u==v:
            x = pos[u]
            CDS.data['x'].append(x[0])
            CDS.data['y'].append(x[1])
            vec = x - np.mean([pos[w] for w in G.neighbors(u)],axis=0)
            a = np.arctan2(vec[1],vec[0])
            CDS.data['a1'].append(a-widthWedge)
            CDS.data['a2'].append(a+widthWedge)
            CDS.data['weight'].append( weight )

In [ ]:
def get_node_data_range(G, nodeAttr):
    vals = [ G.nodes[x][nodeAttr] for x in G.nodes() ]
    return np.min(vals), np.max(vals)

def get_node_color_label_map(G):
    vals = [ x for x in G.nodes() ]
    _, i = np.unique(vals, return_index=True)
    vMap=np.array([ vals[x] for x in np.sort(i)])
    return vMap

def format_graph_nodes_by_palette(G, palette, nullColour='#FFFFFF'):
    # Set the node properties as additional entries in the graph.
    # Should I wrap palette around for text encodings that has more types than the number of colours in palette
    pMax = len(palette)
    nodeColors={}
    vMap=get_node_color_label_map(G)
    
    for n in G.nodes():
        i = np.where(vMap==n)[0][0]
        if i>=pMax:
            nodeColors[n] = nullColour
        else:
            nodeColors[n] = palette[ i ]
    nx.set_node_attributes(G, nodeColors, "node_color")

In [ ]:
colourPaletteCat = ['#FF0000'] + list( bokehPalettes.Colorblind[8] ) + ['#666666']
format_graph_nodes_by_palette(G, colourPaletteCat)
#format_graph_edges_by_palette(G, colourPaletteLin)

In [ ]:
# SAVE
#output_file('Sample_Application.html',mode='inline',root_dir=None)
output_notebook()

# = = = = = = = = = = = = = = = = = = = = = = = = = = =
# = = General settings.
# = = = = = = = = = = = = = = = = = = = = = = = = = = =
plotWidth=600 ; plotHeight=400

# = = = = = = = = = = = = = = = = = = = = = = = = = = =
# = = Graph
# = = = = = = = = = = = = = = = = = = = = = = = = = = =

# = = = Plot Prep
figA = bokehPlotting.figure(plot_width=plotWidth, plot_height=plotHeight,
              tools=["pan","wheel_zoom", "tap", "reset", "save"],
              title="Overview")
figA.toolbar.active_scroll = figA.select_one(bokehModels.WheelZoomTool)
figA.title.text = "Graph Search Demonstration"

# = = = Plot most of graph
rendererGraph = bokehPlotting.from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))
rendererGraph.node_renderer.glyph = bokehModels.Circle(size=15, fill_color='node_color')
rendererGraph.edge_renderer.glyph = bokehModels.MultiLine(line_color='grey', line_alpha=0.8, line_width=1)
rendererGraph.node_renderer.selection_glyph = bokehModels.Circle(size=20, fill_color='node_color')

# = = = Plot self-edge
sourceSelfEdge = bokehModels.ColumnDataSource(data=dict(x=[], y=[], a1=[], a2=[], weight=[]))
#glyphSelfEdge = bokehModels.Circle(x="x", y="y", size=30, fill_alpha=0.0,
#                                     line_color='grey', line_alpha=0.8, line_width=1)
glyphSelfEdge = bokehModels.AnnularWedge(x="x", y="y", inner_radius=0, outer_radius=20, outer_radius_units='screen',
                                         start_angle="a1", end_angle="a2", fill_alpha=0.0,
                                         line_color='grey', line_alpha=0.8, line_width=1)

arrange_self_edges(G, sourceSelfEdge, rendererGraph.layout_provider.graph_layout)

# = = = Layers renderers
sourceGraph = rendererGraph.node_renderer.data_source
rendererSelfEdges = figA.add_glyph(sourceSelfEdge, glyphSelfEdge)
figA.renderers.append(rendererGraph)

# = = = Hover Tools
edge_hover_tool = bokehModels.HoverTool(tooltips=[("weight", "@weight")], renderers=[rendererGraph.edge_renderer,rendererSelfEdges])
figA.add_tools(edge_hover_tool)
node_hover_tool = bokehModels.HoverTool(tooltips=[("index", "@index")], renderers=[rendererGraph.node_renderer])
figA.add_tools(node_hover_tool)
#p.add_tools(bokehModels.HoverTool(tooltips=tooltips, renderers=[rendererA,rendererB]))

# = = = Colour Bar
colourList = get_node_color_label_map(G)
palette = colourPaletteCat
if len(colourList) > len(palette):
    colourList = colourList[:len(palette)]
elif len(palette) > len(colourList):
    palette = colourPaletteCat[:len(colourList)]
colourMapper = bokehModels.mappers.CategoricalColorMapper(palette=palette, factors=colourList)
colourBar = bokehModels.ColorBar(name='SegID', color_mapper=colourMapper, label_standoff=12)
figA.add_layout(colourBar, 'right')        

show(figA)